In [31]:
import pandas
import requests
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

data = pandas.read_csv("reviews.csv")

In [32]:
X = data["review"]
y = data["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
X_train_text = X_train
X_train_text = X_train_text.reset_index(drop=True)
vec = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
X_train = vec.fit_transform(X_train)
X_test = vec.transform(X_test)

clf = LinearSVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, pos_label="pos"))


report_2 = classification_report(y_test, y_pred)
print(report_2)

0.8206666666666667
0.8181203515889114
              precision    recall  f1-score   support

         neg       0.82      0.83      0.82       758
         pos       0.82      0.82      0.82       742

    accuracy                           0.82      1500
   macro avg       0.82      0.82      0.82      1500
weighted avg       0.82      0.82      0.82      1500



In [33]:
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, pos_label="pos"))

0.648
0.6562499999999999


In [34]:
df_vysledek = pandas.DataFrame(y_test, columns=["y_test"])
df_vysledek["y_pred"] = y_pred
df_vysledek["text"] = X_train_text
df_vysledek

,y_test,y_pred,text
0,NaN,pos,"they say 100% compatible but is not , because ..."
1,NaN,pos,what a disaster . stay away from this film . i...
2,NaN,neg,first off anyone who gave this book five stars...
3,NaN,pos,smells divine ! this is a powder-sweet and ref...
4,NaN,pos,as a character movie this is largely a failure...
...,...,...,...
1495,NaN,neg,i 'm not sure what to call this cd other than ...
1496,NaN,pos,"save your time , money and patience . this may..."
1497,NaN,neg,i ca n't even comment on the actual system wor...
1498,NaN,neg,i bought this camera to take pictures of my fa...
